<a href="https://colab.research.google.com/github/XuRui314/MIT_6.036_homework_zxr/blob/main/Tree_Based_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

这个是对应博客上决策树搭建与训练的部分，以及随机森林和GBDT的对应实现 :)



In [25]:
import seaborn as sns
import numpy as np
import pandas as pd

In [7]:
data = sns.load_dataset("iris")
data

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


决策树的组成部分其实就是决策节点、叶子节点、节点间连接关系。下面来定义节点类：

In [3]:
class Node():
  def __init__(self, feature_index = None, threshold = None, left = None, right = None, info_gain = None, value = None):
    # for decision node
    self.feature_index = feature_index
    self.threshold = threshold
    self.left = left
    self.right = right
    self.info_gain = info_gain

    # for leaf node
    self.value = value

然后就是decision tree的类：

## Classification Tree

In [20]:
class DecisionTreeClassifier():
  def __init__(self, min_samples_split = 2, max_depth = 2):
    # initialize the root of the tree
    self.root = None

    # stopping conditions
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth


  def build_tree(self, dataset, curr_depth = 0):
    ''' recursive functoin to bulid the tree'''
    X, Y = dataset[:,:-1], dataset[:,-1]
    num_samples, num_features = np.shape(X)

    # split untill stopping conditions are met
    if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
      # find the best split
      best_split = self.get_best_split(dataset, num_samples, num_features)
      # check if information gain is positive, =0 means the data are actully just in one class
      if best_split["info_gain"] > 0:
        # recur left
        left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
        # recur right
        right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
        # return decision node
        return Node(best_split["feature_index"], best_split["threshold"],
                    left_subtree, right_subtree, best_split["info_gain"])
      
    # compute leaf node
    leaf_value = self.calculate_leaf_value(Y)
    # return leaf node
    return Node(value = leaf_value)


  def get_best_split(self, dataset, num_samples, num_features):
    ''' function to find the best split'''

    # dictionary to store the best split
    best_split = {}
    max_info_gain = -float("inf") # initial value

    # loop over all the feature 
    for feature_index in range(num_features):
      feature_values = dataset[:,feature_index]
      possible_thresholds = np.unique(feature_values)
      # loop over all the feature values presnet in the data
      for threshold in possible_thresholds:
        # get current split
        dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
        # check if childs are not null
        if len(dataset_left)>0 and len(dataset_right)>0:
          y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
          # compute information gain
          curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
          # update the best split if needed
          if curr_info_gain>max_info_gain:
            best_split["feature_index"] = feature_index
            best_split["threshold"] = threshold
            best_split["dataset_left"] = dataset_left
            best_split["dataset_right"] = dataset_right
            best_split["info_gain"] = curr_info_gain
            max_info_gain = curr_info_gain
            
    # return best split
    return best_split


  def split(self, dataset, feature_index, threshold):
    ''' function to find the best split '''
    dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
    dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
    return dataset_left, dataset_right


  def information_gain(self, parent, l_child, r_child, mode="entropy"):
      ''' function to compute information gain '''
      
      weight_l = len(l_child) / len(parent)
      weight_r = len(r_child) / len(parent)
      if mode=="gini":
          gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
      else:
          gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
      return gain
  
  def entropy(self, y):
      ''' function to compute entropy '''
      
      class_labels = np.unique(y)
      entropy = 0
      for cls in class_labels:
          p_cls = len(y[y == cls]) / len(y)
          entropy += -p_cls * np.log2(p_cls)
      return entropy
  
  def gini_index(self, y):
      ''' function to compute gini index '''
      
      class_labels = np.unique(y)
      gini = 0
      for cls in class_labels:
          p_cls = len(y[y == cls]) / len(y)
          gini += p_cls**2
      return 1 - gini
      
  def calculate_leaf_value(self, Y):
      ''' function to compute leaf node '''
      
      Y = list(Y)
      return max(Y, key=Y.count)
  
  def print_tree(self, tree=None, indent=" "):
      ''' function to print the tree '''
      
      if not tree:
          tree = self.root

      if tree.value is not None:
          print(tree.value)

      else:
          print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
          print("%sleft:" % (indent), end="")
          self.print_tree(tree.left, indent + indent)
          print("%sright:" % (indent), end="")
          self.print_tree(tree.right, indent + indent)
  
  def fit(self, X, Y):
      ''' function to train the tree '''
      
      dataset = np.concatenate((X, Y), axis=1)
      self.root = self.build_tree(dataset)
  
  def predict(self, X):
      ''' function to predict new dataset '''
      
      preditions = [self.make_prediction(x, self.root) for x in X]
      return preditions
  
  def make_prediction(self, x, tree):
      ''' function to predict a single data point '''
      
      if tree.value!=None: return tree.value
      feature_val = x[tree.feature_index]
      if feature_val<=tree.threshold:
          return self.make_prediction(x, tree.left)
      else:
          return self.make_prediction(x, tree.right)


      

In [21]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [22]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_2 <= 1.9 ? 0.33741385372714494
 left:setosa
 right:X_3 <= 1.5 ? 0.427106638180289
  left:X_2 <= 4.9 ? 0.05124653739612173
    left:versicolor
    right:virginica
  right:X_2 <= 5.0 ? 0.019631171921475288
    left:X_1 <= 2.8 ? 0.20833333333333334
        left:virginica
        right:versicolor
    right:virginica


In [8]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.9333333333333333

## Regression Tree

## Random Forest

两个随机的地方，一个是数据集是Boostraping生成的，另一个是对每棵树所用特征的随机选择，这个一般取log(num_features)的数量。

In [30]:
from scipy.stats import bootstrap
import random
from random import sample
import math

In [36]:
int(np.log(5) )

1

In [ ]:
class RandomForestClassifier():
  def __init__(self, n_estimators = 100, min_samples_split = 2, max_depth = 2, random_state = 0):
    # initialize the forest
    self.forest = [] # the list of all the roots of trees in the forest

    # other parameters
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_estimators = n_estimators
    self.random_state = random_state

  def build_forest(self, dataset, curr_depth = 0):
    
    num_samples, num_features = np.shape(X)
    feature_indexs = np.arange(num_features).tolist()
    num_select_features = int(np.log(num_features))

    for i in range(n_estimators):
      ''' Bootstrap sampling the dataset '''
      data_sampled = np.random.choice(dataset, size=len(dataset))
      feature_indexs = sample(feature_indexs, num_select_features)

      X, Y = dataset[:,:-1], dataset[:,-1]
      X_sampled = 
      Y_sampled = 
      ''' Random select the features '''
      X_data = 
      Y_data = 

      single_tree = DecisionTreeClassifier(min_samples_split=min_samples_split, max_depth=max_depth)
      single_tree.fit(X_data, Y_data)
      self.forest.append(single_tree) 



  def fit(self, X, Y):
      ''' function to train the tree '''
      
      dataset = np.concatenate((X, Y), axis=1)
      self.forest = self.build_forest(dataset)
  
  def predict(self, X):
      ''' function to predict new dataset '''
      preditions_all = [] # matrix (n_estimators, X.size)
      for i in range(n_estimators):
        preditions.append(self.forest[i].predict(X))
      
      predictions = []

      return preditions 

In [ ]:
classifier = RandomForestClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)

In [ ]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)